In [ ]:
import numpy as np
import tensorflow as tf
from random import randint
import scipy as scipy
import scipy.interpolate as interpolate
import math
from random import *
from sklearn.decomposition import PCA
from sklearn.datasets import load_breast_cancer
from scipy.stats import logistic
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt2

import tflib as lib
import tflib.ops.linear
import tflib.ops.conv2d
import tflib.ops.batchnorm
import tflib.ops.deconv2d
import tflib.save_images
import tflib.plot


Lambda = 10.0
BATCH_SIZE = 56
BATCH_SIZE_2 = 100
number_of_neurons = 200
dimensionality = 30
num_epochs = 20
num_iters = 60
M = 3000
k=2
max_grad_norm = 1000
gamma = 1.0
epsilon = 0.8

In [ ]:
breast = load_breast_cancer(return_X_y=False)
data = []
target = []
rr = np.random.permutation(569)
for x in rr:
    data.append(breast['data'][x])
    target.append(breast['target'][x])
# training data
_train_x = np.array(data)
_train_y = np.array(target)
print(_train_x.shape)
print(_train_y.shape)

In [ ]:
new_lr = tf.placeholder(tf.float32, shape=[], name="new_learning_rate")
batch_size = tf.placeholder(tf.int32, shape=[], name="batch_size")
 
tf_data_x = tf.placeholder(tf.float32, shape=(None,dimensionality)) # узел на который будем подавать аргументы функции
tf_data_y = tf.placeholder(tf.float32, shape=(None,1)) # узел на который будем подавать значения функции

tf_data_w = tf.placeholder(tf.float32, shape=(number_of_neurons,1))
tf_data_B = tf.placeholder(tf.float32, shape=(dimensionality, number_of_neurons))
tf_data_biases = tf.placeholder(tf.float32, shape=(number_of_neurons))
tf_data_P = tf.placeholder(tf.float32, shape=(dimensionality, dimensionality))

tf_data_second = gamma*tf.random_normal([BATCH_SIZE_2, dimensionality])
Lbd = tf.placeholder(tf.float32, shape=[], name="lambda")

w = tf.Variable(tf.random_normal([number_of_neurons,1], stddev=0.35), name="neuron_weights")
B = tf.Variable(tf.random_normal([dimensionality, number_of_neurons], stddev=0.35), name="weights")
biases = tf.Variable(tf.zeros([number_of_neurons]), name="biases")

noise = epsilon*tf.random_normal([batch_size,dimensionality])
logits = tf.matmul(tf.nn.sigmoid(tf.nn.bias_add(tf.matmul(tf_data_x+noise, B), biases)), w)
out_loss = -tf.reduce_mean(tf.multiply(logits, tf_data_y)) + tf.reduce_mean(tf.math.log(1.0+tf.math.exp(logits)))

sigma_prime = tf.multiply(1.0 - tf.nn.sigmoid(tf.nn.bias_add(tf.matmul(tf_data_second, B), biases)), 
                       tf.nn.sigmoid(tf.nn.bias_add(tf.matmul(tf_data_second, B), biases)))
multiply = tf.stack([BATCH_SIZE_2, 1]) 
w_M_times = tf.tile(tf.transpose(w), multiply)
grad_psi = tf.matmul(tf.multiply(w_M_times, sigma_prime), tf.transpose(B))

new_part = grad_psi

tf_data_sigma_prime = tf.multiply(1.0 - tf.nn.sigmoid(tf.nn.bias_add(tf.matmul(tf_data_second, tf_data_B), tf_data_biases)), 
                       tf.nn.sigmoid(tf.nn.bias_add(tf.matmul(tf_data_second, tf_data_B), tf_data_biases)))
tf_data_w_M_times = tf.tile(tf.transpose(tf_data_w), multiply)
tf_data_grad_psi = tf.matmul(tf.multiply(tf_data_w_M_times, tf_data_sigma_prime), tf.transpose(tf_data_B))

old_part = tf.matmul(tf_data_grad_psi, tf_data_P)

loss2 = out_loss + Lbd*tf.reduce_mean(tf.square(tf.subtract(new_part, old_part)))
loss = 1000*loss2

In [ ]:
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), max_grad_norm)
#optimizer = tf.train.GradientDescentOptimizer(new_lr)
#target = optimizer.apply_gradients(zip(grads, tvars), global_step=tf.contrib.framework.get_or_create_global_step())
target = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5, beta2=0.9).minimize(loss)

In [ ]:
def sigmoid(x):
  return 1. /(1+np.exp(-x))

In [ ]:
scores = []
for part in range(8):
    print("Current part is %d\n" % part)
    train_x = np.concatenate((_train_x[0:(part*65)], _train_x[(part+1)*65:569]), axis=0)
    train_y = np.concatenate((_train_y[0:(part*65)], _train_y[(part+1)*65:569]), axis=0)
    test_x = _train_x[part*65:(part+1)*65]
    test_y = _train_y[part*65:(part+1)*65]
    scaler = preprocessing.StandardScaler().fit(train_x)
    train_x = scaler.transform(train_x)
    test_x = scaler.transform(test_x)

    cur_w = np.random.normal(0, 0.35, (number_of_neurons,1))
    cur_B =  np.random.normal(0, 0.35, (dimensionality, number_of_neurons))
    cur_biases = np.zeros((number_of_neurons))
    cur_P = np.zeros((dimensionality, dimensionality)) 
    O = np.zeros((dimensionality, k)) 
    cur_iter = 0
    
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for epoch in range(num_epochs):
        print("Epoch %d" %(epoch))
        for iteration in range(num_iters):
            offset = (cur_iter % 9)*BATCH_SIZE
            sample_x = np.reshape(train_x[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, dimensionality))
            sample_y = np.reshape(train_y[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, 1))
            sess.run(target, feed_dict={tf_data_x: sample_x, tf_data_y: sample_y, 
                                  tf_data_w: cur_w, tf_data_B: cur_B, tf_data_biases: cur_biases,
                                  tf_data_P: cur_P, Lbd: Lambda, batch_size: BATCH_SIZE})
            cur_iter = cur_iter+1
        reses = []
        outes = []
        for i in range(9):
            offset = (cur_iter % 9)*BATCH_SIZE
            sample_x = np.reshape(train_x[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, dimensionality))
            sample_y = np.reshape(train_y[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, 1))
            res, out = sess.run([loss, out_loss], feed_dict={tf_data_x: sample_x, tf_data_y: sample_y, 
                                  tf_data_w: cur_w, tf_data_B: cur_B, tf_data_biases: cur_biases,
                                  tf_data_P: cur_P, Lbd: Lambda, batch_size: BATCH_SIZE})
            reses.append(res)
            outes.append(out)
            cur_iter = cur_iter+1
        print ("Iter %d: loss: %.4f log-likelihood: %.4f\n" %(cur_iter, np.mean(np.array(reses)), np.mean(np.array(outes))))
        lib.plot.plot('train cost', np.mean(np.array(reses)))
        lib.plot.plot('log-likelihood', np.mean(np.array(outes)))

        cur_w, cur_B, cur_biases = sess.run([w, B, biases], feed_dict={})    
        third_x = np.random.normal(0, 0.35, (M,dimensionality))
        third_sigma_prime = np.multiply(1.0-logistic.cdf(np.add(np.matmul(third_x, cur_B), np.tile(np.reshape(cur_biases,[1, number_of_neurons]),(M,1)))), 
                           logistic.cdf(np.add(np.matmul(third_x, cur_B), np.tile(np.reshape(cur_biases,[1, number_of_neurons]),(M,1)))))
        third_w_M_times = np.tile(np.reshape(cur_w,[1, number_of_neurons]), (M,1))
        third_grad_psi = np.matmul(np.multiply(third_w_M_times, third_sigma_prime), np.transpose(cur_B))
        
        u, s, vh = np.linalg.svd(np.transpose(third_grad_psi), full_matrices=True)
        O = u[:,0:k:1]
        print(s)
        cur_P = np.matmul(O, np.transpose(O))
        tvr = 1-np.sum(np.multiply(s[0:k],s[0:k]))/np.sum(np.multiply(s,s))
        lib.plot.plot('total variance to retain', tvr)
        lib.plot.tick()
        lib.plot.flush()
    reduced_train_x = np.matmul(train_x, O)
    reduced_test_x = np.matmul(test_x, O)
    clf = LogisticRegression(random_state=0).fit(reduced_train_x, train_y)
    score = clf.score(reduced_test_x, test_y)
    scores.append(score)
    print ("Part %d: rate on test %.4f\n" %(part, score))
    test_rgb = []
    for h in test_y:
        if h == 1:
            test_rgb.append('r')
        else:
            test_rgb.append('b')
    plt2.close()
    plt2.scatter(np.transpose(reduced_test_x)[0], np.transpose(reduced_test_x)[1], alpha=0.2, c=np.array(test_rgb))
    plt2.xlabel("1st component")
    plt2.ylabel("2nd component")
    plt2.show()
    sess.close()
    print(cur_P)
print ("Average rate on test %.4f\n" %(np.mean(np.array(scores))))